# Travel Agent 🌎✈️
### Multi-Agent Travel Planning System with ML Models

This notebook provides an interactive travel planning system that generates personalized itineraries, cost estimates, and venue recommendations using machine learning models.

**Features:**
- ML-powered cost predictions for food, lodging, entertainment, and transportation
- Personalized venue recommendations
- Budget-aware planning with customizable dining preferences
- Interactive widgets for easy input

## 📦 Install Dependencies
Install required Python packages

In [0]:
%pip install pandas pyarrow numpy duckdb xgboost joblib scikit-learn colour
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## 📥 Import Libraries

In [0]:
import pandas as pd
import pyarrow
import xgboost
import joblib
import numpy as np
import pickle
import joblib
import os
import json
import math
import logging
from sklearn.preprocessing import normalize
import warnings

warnings.filterwarnings('ignore')
print("✅ Libraries imported successfully")



✅ Libraries imported successfully


In [0]:
# Setup the configuration
storage_account = "lab94290"
sas_token = "" # Replace with your SAS token
container = "submissions"
group = "Gil_Murad_Guy"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", sas_token)

# Define the file path (using the ABFS driver)
DIRECTORY_PATH = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{group}/"

# Try to list the files to see if it works
display(dbutils.fs.ls(DIRECTORY_PATH))

import pickle
import os

# Define the cloud path for models
cloud_folder = DIRECTORY_PATH + "models/"

# A temporary spot on the Databricks cluster (Local Driver)
local_folder_dbutils = "file:/tmp/models/"  # HAS prefix (for dbutils)
local_folder_python  = "/tmp/models/"       # NO prefix (for python)

# Copy using dbutils (Needs 'file:')
# create the directory first to be safe
dbutils.fs.mkdirs(local_folder_dbutils) 
dbutils.fs.cp(cloud_folder, local_folder_dbutils, recurse=True)

# List using Python (NO 'file:')
print("✅ Files on driver:")
print(os.listdir(local_folder_python))  # <--- Use the path without 'file:'

# Set paths for models
MODELS_PATH = local_folder_python
display(dbutils.fs.ls(MODELS_PATH))

# Read the CSV file
import io
csv_content = dbutils.fs.head(DIRECTORY_PATH + "transportation data/states.csv", 1000000)
df = pd.read_csv(io.StringIO(csv_content))

# Create the dictionary: Set 'state' as keys and 'abbreviation' as values
state_to_abbrev_map = pd.Series(df.abbreviation.values, index=df.state).to_dict()
abbrev_to_state_map = {v: k for k, v in state_to_abbrev_map.items()} # Swapping pairs

# Print the result
print(state_to_abbrev_map)
print(abbrev_to_state_map)
print(len(state_to_abbrev_map.items()))

path,name,size,modificationTime
abfss://submissions@lab94290.dfs.core.windows.net/Gil_Murad_Guy/entertainment data/,entertainment data/,0,1769610850000
abfss://submissions@lab94290.dfs.core.windows.net/Gil_Murad_Guy/food data/,food data/,0,1769610925000
abfss://submissions@lab94290.dfs.core.windows.net/Gil_Murad_Guy/models/,models/,0,1769786575000
abfss://submissions@lab94290.dfs.core.windows.net/Gil_Murad_Guy/transportation data/,transportation data/,0,1769610807000


✅ Files on driver:
['lodging_cost_predictor_v1.pkl', 'food_cost_predictor_v1.pkl', 'recommendation_engine.pkl', 'entertainment_cost_model.pkl', '.lodging_cost_predictor_v1.pkl.crc', '.food_cost_predictor_v1.pkl.crc', '.recommendation_engine.pkl.crc', '.entertainment_cost_model.pkl.crc']


path,name,size,modificationTime
dbfs:/tmp/models/entertainment_cost_model.pkl,entertainment_cost_model.pkl,485775,1769876395000
dbfs:/tmp/models/food_cost_predictor_v1.pkl,food_cost_predictor_v1.pkl,2087795,1769876395000
dbfs:/tmp/models/lodging_cost_predictor_v1.pkl,lodging_cost_predictor_v1.pkl,146843,1769876395000
dbfs:/tmp/models/recommendation_engine.pkl,recommendation_engine.pkl,1800342,1769876395000


{'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}
{'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizon

## 🏗️ Define Agent Classes

### Food Cost Agent

In [0]:
# USA Food Plan Data (Monthly Costs)
# Updated to include approximate child costs (Avg of age 6-11 brackets)
GROCERY_PLANS = {
    "low": {"male": 371.0, "female": 323.0, "child": 240.0},
    "moderate": {"male": 465.0, "female": 392.0, "child": 300.0},
    "liberal": {"male": 566.0, "female": 499.0, "child": 375.0}
}


class FoodCostAgent:
    def __init__(self, model_path="sub_agents/models/food_cost_predictor_v1.pkl"):
        """
        Initializes the agent by loading the v1 XGBoost model and USA/Index maps.
        """
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"❌ Error: Could not find '{model_path}'.")

        print(f"Loading Food Model from {model_path}...")
        try:
            artifacts = joblib.load(model_path)

            # Unpack Artifacts
            self.model = artifacts["xgb_model"]
            self.features = artifacts["model_columns"]
            self.grocery_map = artifacts["grocery_index_map"]

            # Handle naming variation
            self.state_map = artifacts.get("state_map_helper", artifacts.get("state_map", {}))

            # Calculate National Avg if not explicitly saved
            self.national_avg_index = artifacts.get("national_avg_index", np.mean(list(self.grocery_map.values())))

            # Extract supported cuisines for validation
            self.known_cuisines = [
                c.replace("clean_cuisine_", "")
                for c in self.features if c.startswith("clean_cuisine_")
            ]
            print("✅ Food Model loaded successfully.")

        except Exception as e:
            print(f"❌ Failed to load food model artifacts: {e}")
            raise e

    def get_supported_cuisines(self):
        return self.known_cuisines

    def _get_state_code(self, user_input):
        """Standardizes input to 2-letter state code."""
        clean = str(user_input).lower().strip()
        if len(clean) == 2:
            return clean.upper()
        for name, code in self.state_map.items():
            if name in clean:
                return code
        return "Unknown"

    def predict_cost(self,
                     location,
                     days,
                     demographics=None,
                     eating_out_ratio=0.5,
                     cuisine="American",
                     vibe_rating=4.5,
                     budget_level="moderate"):
        """
        Calculates food costs combining:
        1. XGBoost Model (Restaurants)
        2. USDA Food Plans (Groceries) with Demographic breakdown
        3. Custom Eating Out Ratio

        Args:
            location (str): State or location name.
            days (int): Duration of the trip.
            demographics (dict): {'males': int, 'females': int, 'children': int}. Defaults to 1 Male.
            eating_out_ratio (float): 0.0 (all grocery) to 1.0 (all restaurant). Default 0.5.
            cuisine (str): Preferred cuisine type.
            vibe_rating (float): Expected quality/rating of restaurants.
            budget_level (str): 'low', 'moderate', or 'liberal'.
        """
        state_code = self._get_state_code(location)

        # Default demographics if None provided
        if demographics is None:
            demographics = {"males": 1, "females": 0, "children": 0}

        total_people = sum(demographics.values())
        if total_people == 0:
            total_people = 1  # Fallback
            demographics["males"] = 1

        input_df = pd.DataFrame(0, index=[0], columns=self.features)
        input_df['rating'] = vibe_rating
        input_df['review_count'] = 150

        # One-Hot Encoding: State
        if f"state_{state_code}" in input_df.columns:
            input_df[f"state_{state_code}"] = 1

        # One-Hot Encoding: Cuisine
        target_cuisine = "Other"
        for k in self.known_cuisines:
            if k.lower() == cuisine.lower():
                target_cuisine = k
                break

        if f"clean_cuisine_{target_cuisine}" in input_df.columns:
            input_df[f"clean_cuisine_{target_cuisine}"] = 1

        try:
            pred_price = float(self.model.predict(input_df)[0])
            price_restaurant_meal = max(pred_price, 7.00)
        except Exception as e:
            print(f"  [ML Warning] Prediction failed, using fallback. Error: {e}")
            price_restaurant_meal = 15.00

        plan_key = budget_level.lower()
        if plan_key not in GROCERY_PLANS:
            plan_key = "moderate"

        plan_costs = GROCERY_PLANS[plan_key]

        # Calculate Monthly Base based on specific demographics
        monthly_base_national = (
                (demographics.get("males", 0) * plan_costs["male"]) +
                (demographics.get("females", 0) * plan_costs["female"]) +
                (demographics.get("children", 0) * plan_costs["child"])
        )

        # Convert to Weekly Base
        weekly_base_national = monthly_base_national / 4.33

        # Apply State Index Multiplier
        state_idx = float(self.grocery_map.get(state_code, self.national_avg_index))
        state_multiplier = state_idx / 100.0

        # Full Grocery Cost (assuming 100% cooking)
        weekly_grocery_local_total = weekly_base_national * state_multiplier

        weeks = days / 7.0

        # Validate ratio (clamp between 0 and 1)
        eating_out_ratio = max(0.0, min(1.0, eating_out_ratio))

        # Assume 21 meals per week (3 per day)
        total_meals = 21
        meals_restaurant = total_meals * eating_out_ratio

        # Grocery Factor Logic:
        # If eating out 100%, grocery is not 0% (snacks, water, breakfast), floor at 10%
        # If eating out 0%, grocery is 100%
        grocery_utilization = max(0.1, 1.0 - eating_out_ratio)

        cost_user_custom = (price_restaurant_meal * meals_restaurant * weeks * total_people) + \
                           (weekly_grocery_local_total * weeks * grocery_utilization)


        # Saver: Eat out 10% of time (approx 2 meals/week)
        cost_saver = (price_restaurant_meal * (21 * 0.1) * weeks * total_people) + \
                     (weekly_grocery_local_total * weeks * 0.9)

        # Balanced: Eat out 30% of time (approx 6 meals/week)
        cost_balanced = (price_restaurant_meal * (21 * 0.3) * weeks * total_people) + \
                        (weekly_grocery_local_total * weeks * 0.7)

        # Foodie: Eat out 70% of time (approx 15 meals/week)
        cost_foodie = (price_restaurant_meal * (21 * 0.7) * weeks * total_people) + \
                      (weekly_grocery_local_total * weeks * 0.3)

        return {
            "total_cost": float(round(cost_user_custom, 2)),
            "parameters": {
                "ratio_used": eating_out_ratio,
                "demographics": demographics
            },
            "breakdown": {
                "unit_restaurant_meal": float(round(price_restaurant_meal, 2)),
                "weekly_grocery_full_cost": float(round(weekly_grocery_local_total, 2)),
                "state_index": state_idx
            },
            "food_options": {
                "grocery_heavy": float(round(cost_saver, 2)),
                "balanced": float(round(cost_balanced, 2)),
                "restaurant_only": float(round(cost_foodie, 2))
            }
        }

print("✅ FoodCostAgent class defined")

✅ FoodCostAgent class defined


### Lodging Cost Agent

In [0]:
import pandas as pd
import joblib
import os
import numpy as np


class LodgingCostAgent:
    def __init__(self, model_path="sub_agents/models/lodging_cost_predictor_v1.pkl"):
        # Load the Brain
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"❌ Error: Could not find '{model_path}'.")

        print(f"Loading Lodging Model from {model_path}...")
        try:
            artifacts = joblib.load(model_path)
            self.model = artifacts["lodging_model"]
            self.features = artifacts["model_columns"]  # The specific columns XGBoost expects

            # The model was trained with 'clean_state_X'. We need to know what 'X' is.
            # We extract the list of states the model actually learned as distinct features.
            self.known_states_in_model = [
                col.replace("clean_state_", "")
                for col in self.features
                if col.startswith("clean_state_")
            ]

            print(f"✅ Lodging Model loaded. It knows {len(self.known_states_in_model)} distinct states.")

        except Exception as e:
            print(f"❌ Failed to load lodging artifacts: {e}")
            raise e

        # Helps us bridge the gap if user says "NY" but model knows "New York" (or vice versa)
        self.abbr_to_name = abbrev_to_state_map
        # Create reverse map too
        self.name_to_abbr = {v.lower(): k for k, v in self.abbr_to_name.items()}

    def _resolve_state_feature(self, user_input):
        """
        Matches user input (e.g. 'NY') to the exact column name in the model (e.g. 'clean_state_New York').
        Returns the suffix string or None.
        """
        raw = str(user_input).strip()

        # Exact Match (e.g. Model has 'NY', input is 'NY')
        if raw in self.known_states_in_model:
            return raw

        # Try Full Name (e.g. Model has 'New York', input is 'NY')
        if raw.upper() in self.abbr_to_name:
            full_name = self.abbr_to_name[raw.upper()]
            if full_name in self.known_states_in_model:
                return full_name

        # Try Abbr (e.g. Model has 'NY', input is 'New York')
        if raw.lower() in self.name_to_abbr:
            abbr = self.name_to_abbr[raw.lower()]
            if abbr in self.known_states_in_model:
                return abbr

        # Case-insensitive scan
        for s in self.known_states_in_model:
            if raw.lower() == s.lower():
                return s

        return None

    def predict_cost(self, state, travelers, nights, room_type="Entire Home/Apt", rating=4.8, luxury_level="Moderate"):
        """
        Predicts using the EXACT XGBoost model structure from your training code.
        """

        # Map Inputs to Training Features
        # Your training code used these defaults:
        # defaults = {"rating": 4.5, "review_count": 0, "amenities_count": 5, "host_score": 4.5, "desc_length": 100}

        amenities = 5
        host_score = 4.5
        desc_len = 100
        review_count = 50  # Assume established listing

        # Adjust based on 'Luxury Level' vibe
        if "cheap" in luxury_level.lower():
            amenities = 3
            desc_len = 50
            if room_type == "Entire Home/Apt": room_type = "Private Room"  # Downgrade logic

        elif "luxury" in luxury_level.lower():
            amenities = 25
            host_score = 4.9
            desc_len = 800
            review_count = 150

        # Build the Input Vector (DataFrame)
        # We start with 0s for everything
        input_df = pd.DataFrame(0, index=[0], columns=self.features)

        # Set Numerical Columns (Names match your training code exactly)
        input_df['guests'] = travelers
        input_df['rating'] = rating
        input_df['review_count'] = review_count
        input_df['amenities_count'] = amenities
        input_df['host_score'] = host_score
        input_df['desc_length'] = desc_len

        # Set One-Hot Encoded Room Type
        # Training used: pd.get_dummies(..., drop_first=True)
        # We map standard terms to the likely column names
        target_room = "Private Room"  # Default
        r_lower = room_type.lower()

        if "entire" in r_lower or "home" in r_lower or "apt" in r_lower:
            target_room = "Entire Home/Apt"
        elif "hotel" in r_lower:
            target_room = "Hotel"
        elif "shared" in r_lower:
            target_room = "Shared Room"

        # The training code generates columns like "room_type_Hotel", "room_type_Private Room"
        room_col = f"room_type_{target_room}"
        if room_col in input_df.columns:
            input_df[room_col] = 1

        # Set One-Hot Encoded State (The Critical Fix)
        state_suffix = self._resolve_state_feature(state)

        if state_suffix:
            state_col = f"clean_state_{state_suffix}"
            input_df[state_col] = 1
            # print(f"  [Lodging] Matched '{state}' to model column '{state_col}'")
        else:
            # If state not found, we leave all state columns 0.
            # In get_dummies(drop_first=True), 0s everywhere means "Reference Category" (often 'Other')
            # print(f"  [Lodging] State '{state}' unknown to model. Using baseline/Other.")
            pass

        # Predict
        try:
            # XGBoost predict
            price_per_night = float(self.model.predict(input_df)[0])

            # Sanity check: If model predicts negative or near-zero (possible with regression)
            min_price = 10.0 * travelers
            price_per_night = max(price_per_night, min_price)

        except Exception as e:
            # print(f"  [Lodging Error] {e}")
            price_per_night = 80.0 * travelers  # Emergency fallback

        total_cost = price_per_night * nights

        return {
            "total_cost": float(round(total_cost, 2)),
            "per_night": float(round(price_per_night, 2)),
            "details": {
                "room_type": target_room,
                "state_feature": state_suffix if state_suffix else "Other/Baseline"
            }
        }

print("✅ LodgingCostAgent class defined")

✅ LodgingCostAgent class defined


### Entertainment Agent

In [0]:
class EntertainmentAgent:
    def __init__(self,
                 model_path="sub_agents/models/entertainment_cost_model.pkl",
                 city_data_path="city_profiles.csv"):

        # Load the "Brain" (Model + Config)
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"❌ Error: Could not find model at '{model_path}'.")

        print(f"Loading Entertainment Agent from {model_path}...")
        try:
            with open(model_path, 'rb') as f:
                artifacts = pickle.load(f)

            self.model = artifacts["model"]
            self.features = artifacts["features"]
            self.style_map = artifacts["travel_style_map"]

            self.numeric_cols = [f for f in self.features if
                                 f not in ['travel_style_encoded', 'num_people', 'num_days']]

            print("✅ Entertainment Model & Config loaded.")
        except Exception as e:
            print(f"❌ Failed to load model artifacts: {e}")
            raise e

        # Load City Knowledge Base
        if dbutils.fs.ls(city_data_path):
            try:
                if city_data_path.endswith('.parquet'):
                    self.city_db = spark.read.parquet(city_data_path).toPandas()
                else:
                    self.city_db = spark.read.csv(city_data_path, header=True, inferSchema=True).toPandas()

                self.city_db['city_lower'] = self.city_db['city'].astype(str).str.lower().str.strip()
                self.city_db['state_lower'] = self.city_db['state'].astype(str).str.lower().str.strip()
                print(f"✅ Loaded city profiles for {len(self.city_db)} cities.")

            except Exception as e:
                print(f"⚠️ Warning: Could not load city profiles: {e}")
                self.city_db = pd.DataFrame()
        else:
            print(f"⚠️ Warning: City data not found at {city_data_path}. Predictions will rely on averages.")
            self.city_db = pd.DataFrame()

        # State Mapping Dictionary
        self.state_map = {k.lower(): v.lower() for k, v in state_to_abbrev_map.items()}

    def _get_city_features(self, city, state):
        if self.city_db.empty:
            return None

        target_city = city.lower().strip()
        target_state = state.lower().strip()

        # Map full state name to abbreviation if needed (e.g. "Nevada" -> "nv")
        if target_state in self.state_map:
            target_state = self.state_map[target_state]

        # Candidate names
        candidates = [target_city]
        if "new york" in target_city and "city" not in target_city:
            candidates.append(target_city + " city")
        elif "washington" in target_city and "dc" not in target_city:
            candidates.append("washington")

        for name in candidates:
            # Exact Match (City + State)
            if target_state:
                match = self.city_db[
                    (self.city_db['city_lower'] == name) &
                    (self.city_db['state_lower'] == target_state)
                    ]
                if not match.empty:
                    return match.iloc[0].to_dict()

            # Loose Match (City Only)
            match = self.city_db[self.city_db['city_lower'] == name]
            if not match.empty:
                return match.iloc[0].to_dict()

        # State Fallback
        if target_state:
            state_neighbors = self.city_db[self.city_db['state_lower'] == target_state]
            if not state_neighbors.empty:
                print(f"  [Ent ML] City '{city}' not found. Using average for state '{target_state.upper()}'.")
                return state_neighbors.mean(numeric_only=True).fillna(0).to_dict()

        return None

    def _get_national_averages(self):
        if self.city_db.empty:
            return {'venue_count': 50, 'avg_price': 30.0}
        return self.city_db.mean(numeric_only=True).to_dict()

    def predict_cost(self, city, state, travelers, days, budget_level="Moderate"):
        # Get City/State Stats
        city_stats = self._get_city_features(city, state)

        if not city_stats:
            print(f"  [Ent ML] Location '{city}, {state}' not found. Using national averages.")
            city_stats = self._get_national_averages()

        # Map Travel Style
        style_key = "medium"
        b_lower = budget_level.lower()
        if "cheap" in b_lower:
            style_key = "budget"
        elif "luxury" in b_lower:
            style_key = "luxury"
        elif "expensive" in b_lower:
            style_key = "expensive"

        encoded_style = self.style_map.get(style_key, 1)

        # Construct Input Vector
        input_data = {}

        for feature in self.features:
            if feature in city_stats:
                input_data[feature] = city_stats[feature]
            elif feature == "num_people":
                input_data[feature] = travelers
            elif feature == "num_days":
                input_data[feature] = days
            elif feature == "travel_style_encoded":
                input_data[feature] = encoded_style
            else:
                input_data[feature] = 0.0

        input_df = pd.DataFrame([input_data])
        input_df = input_df[self.features]

        # Predict
        try:
            predicted_cost = float(self.model.predict(input_df)[0])
            min_cost = 5.0 * travelers * days
            final_cost = max(predicted_cost, min_cost)
            return float(round(final_cost, 2))

        except Exception as e:
            print(f"  [Ent ML Error] Prediction failed: {e}")
            base = 30.0 * (0.6 if style_key == 'budget' else 2.5 if style_key == 'luxury' else 1.0)
            return float(round(base * travelers * days, 2))

print("✅ EntertainmentAgent class defined")

✅ EntertainmentAgent class defined


### Transport Agent

In [0]:
class TransportAgent:
    def __init__(self, state_json_path="sub_agents/models/transportation_state.json",
                 city_csv_path="sub_agents/models/transportation_data.csv"):
        # Source of Truth for Geographic Mapping
        self.state_map = abbrev_to_state_map

        # Bi-directional Normalizer: Ensures "Arizona", "ARIZONA", and "AZ" all map to "AZ"
        self.name_to_abbr = {v.upper(): k.upper() for k, v in self.state_map.items()}
        
        for abbrev in self.state_map.keys():
            self.name_to_abbr[abbrev.upper()] = abbrev.upper()

        # Load State Data (JSON) from Azure storage
        self.state_lookup = {}
        try:
            raw = spark.read.option("multiline", "true").json(state_json_path).toPandas()
            print(f"✅ State JSON loaded. Standardized {len(raw)} states.")
            for r in raw.to_dict('records'):
                # Standardize JSON key to the 2-letter abbreviation
                raw_state = r["state"].upper().strip()
                ls_type = r["lifestyle"].lower().strip()
                cost = float(r["cost"])

                # Convert "Arizona" -> "AZ". If it's already "AZ", it stays "AZ".
                clean_state = self.name_to_abbr.get(raw_state, raw_state)
                self.state_lookup[(clean_state, ls_type)] = cost
        except Exception as e:
            print(f"❌ Failed to load JSON: {e}")
            import traceback
            traceback.print_exc()

        # Load City Data (CSV) from Azure storage
        self.city_df = None
        try:
            df = spark.read.csv(city_csv_path, header=True, inferSchema=True).toPandas()

            # Force columns to be a list of strings and clean them
            df.columns = [str(c).strip().lower() for c in df.columns.tolist()]

            # Drop junk rows and fill numeric holes
            threshold = len(df.columns) // 2
            df = df.dropna(thresh=threshold)

            # Impute numeric columns only
            numeric_cols = df.select_dtypes(include=['number']).columns
            df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

            # Robust Column Cleaning
            if 'abbreviation' in df.columns and 'city' in df.columns:
                # Use .str accessor for safety on the series data itself
                df['city_clean'] = df['city'].astype(str).str.lower().str.strip()
                df['state_clean'] = df['abbreviation'].astype(str).str.upper().str.strip()
                self.city_df = df
                print(f"✅ City CSV loaded. Standardized {len(df)} profiles.")
            else:
                print(f"⚠️ CSV missing required columns. Found: {df.columns.tolist()}")

        except Exception as e:
            print(f"❌ Failed to load CSV: {e}")

    def _city_transport_cost(self, row, days, travelers, travel_mode):
        def safe_float(val, default):
            try:
                if isinstance(val, float) and math.isnan(val): return default
                return float(val) if val is not None else default
            except:
                return default

        ticket = safe_float(row.get("one-way ticket (local transport)"), 2.50)
        monthly = safe_float(row.get("monthly public transport pass (regular price)"), 80.00)
        taxi_start = safe_float(row.get("taxi start (standard tariff)"), 3.50)
        taxi_dist = safe_float(row.get("taxi 1 km (standard tariff)") or row.get("taxi 1 mile (standard tariff)"), 2.00)

        taxis_needed = math.ceil(travelers / 4)
        if days >= 30:
            ticket_cost = monthly * travelers
        else:
            mult = 4 if travel_mode == 0 else 2
            ticket_cost = ticket * mult * travelers * days

        if travel_mode == 0: return round(ticket_cost, 2)
        if travel_mode == 1:
            taxi_cost = (taxi_start + taxi_dist * 5) * taxis_needed * (days / 2)
            return round(ticket_cost + taxi_cost, 2)

        taxi_cost = (taxi_start + taxi_dist * 10) * 2 * taxis_needed * days
        return round(taxi_cost, 2)

    def predict_cost(self, state, days, travelers, travel_mode, city=None, lifestyle="typical"):
        # Normalize identifiers
        st_input = str(state).strip().upper()
        st_abbr = self.name_to_abbr.get(st_input, st_input)
        ls = str(lifestyle).lower().strip()

        # Try City Match
        if city and self.city_df is not None:
            city_clean = city.lower().strip()
            match = self.city_df[
                (self.city_df["city_clean"] == city_clean) &
                (self.city_df["state_clean"] == st_abbr)
                ]
            if not match.empty:
                return self._city_transport_cost(match.iloc[0], days, travelers, travel_mode)

        # State Fallback
        lookup_key = (st_abbr, ls)
        cost = self.state_lookup.get(lookup_key)

        # The NaN/None Trap
        if cost is None:
            return round(15.0 * days * travelers, 2)

        if isinstance(cost, float) and math.isnan(cost):
            return round(15.0 * days * travelers, 2)

        # Calculation
        try:
            daily_rate = float(cost) / 30
            return round(daily_rate * days * travelers, 2)
        except Exception as e:
            return round(15.0 * days * travelers, 2)
        
print("✅ TransportAgent class defined")

✅ TransportAgent class defined


### Recommendation Agent

In [0]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.preprocessing import normalize


class RecommendationAgent:
    def __init__(self, model_path="sub_agents/models/recommendation_engine.pkl"):
        if not os.path.exists(model_path):
            print(f"⚠️ Recommendation model not found at {model_path}")
            self.model_loaded = False
            return

        try:
            with open(model_path, 'rb') as f:
                data = pickle.load(f)

            self.venues_df = data['venues_df']

            # Use float32 to save memory and reduce complexity
            raw_matrix = np.array(data['feature_matrix'], dtype=np.float32)

            # This fixes the "overflow" issue if we have a price of $1,000,000
            raw_matrix[raw_matrix > 10000.0] = 10000.0

            # Replace NaN/Infinity with 0 or max cap
            raw_matrix = np.nan_to_num(raw_matrix, nan=0.0, posinf=10000.0, neginf=-10000.0)

            # Compresses all vectors to length of 1.0
            self.feature_matrix = normalize(raw_matrix, axis=1, norm='l2')

            self.feature_columns = data['feature_columns']
            self.config = data['config']

            if 'price_avg' not in self.venues_df.columns:
                self.venues_df['price_avg'] = 50.0

            self.model_loaded = True
            print("  [ML] RecommendationAgent loaded. Matrix shape:", self.feature_matrix.shape)

        except Exception as e:
            print(f"❌ Error loading RecommendationAgent: {e}")
            self.model_loaded = False

    def _create_preference_vector(self, preferences):
        """Converts user text preferences into a mathematical vector."""
        vector = np.zeros(len(self.feature_columns), dtype=np.float32)
        cols = self.feature_columns
        known_cats = self.config.get('CATEGORIES', [])

        # Interests
        user_interests = preferences.get('interests', [])
        matched = False
        for interest in user_interests:
            for cat in known_cats:
                if cat in interest.lower() or interest.lower() in cat:
                    col_name = f'cat_{cat}'
                    if col_name in cols:
                        vector[cols.index(col_name)] = 1.0
                        matched = True

        # Fallback
        if not matched:
            for cat in known_cats:
                col_name = f'cat_{cat}'
                if col_name in cols:
                    vector[cols.index(col_name)] = 0.2

        # Budget
        budget = preferences.get('budget', 'medium').lower()
        mapping = {'cheap': 'budget', 'expensive': 'expensive', 'luxury': 'luxury'}
        budget_key = next((k for k, v in mapping.items() if k in budget), 'medium')

        price_col = f'price_{budget_key}'
        if price_col in cols:
            vector[cols.index(price_col)] = 1.0

        # Group
        travelers = preferences.get('travelers', 1)
        group_type = 'all' if travelers > 2 else 'adults'
        aud_col = f'aud_{group_type}'
        if aud_col in cols:
            vector[cols.index(aud_col)] = 1.0

        # Normalize vector
        vector = vector.reshape(1, -1)
        vector = normalize(vector, axis=1, norm='l2')
        return vector

    def recommend(self, city, state, preferences, top_n=5):
        if not self.model_loaded:
            return []

        city_mask = (self.venues_df['city'].str.lower() == city.lower())
        state_mask = (self.venues_df['state'].str.lower() == state.lower()) if state else city_mask

        location_mask = city_mask & state_mask if state else city_mask

        # Fallback
        if location_mask.sum() < 2 and state:
            print(f"  [RecAgent] Few results for {city}. Expanding search to all of {state}.")
            location_mask = state_mask

        if location_mask.sum() == 0:
            return []

        current_indices = np.where(location_mask)[0]
        max_spend = preferences.get('max_spend', None)

        if max_spend is not None and not np.isnan(max_spend):
            limit = max_spend * 1.2
            budget_mask = (self.venues_df['price_avg'] <= limit)
            combined_mask = location_mask & budget_mask

            if combined_mask.sum() > 0:
                current_indices = np.where(combined_mask)[0]
            else:
                print(f"  [RecAgent] Budget ${max_spend} too low. Showing cheapest.")

        # Prepare data
        relevant_features = self.feature_matrix[current_indices].copy()
        relevant_venues = self.venues_df.iloc[current_indices].copy()

        # Explicitly cap values > 10000 again on this subset
        relevant_features[relevant_features > 10000.0] = 10000.0
        relevant_features = np.nan_to_num(relevant_features, nan=0.0, posinf=10000.0, neginf=-10000.0)

        user_vector = self._create_preference_vector(preferences)
        user_vector = np.nan_to_num(user_vector, nan=0.0)

        try:
            # Safe Dot Product
            scores = np.dot(relevant_features, user_vector.T).flatten()
        except Exception as e:
            print(f"⚠️ [RecAgent] Matrix Math Failed ({e}). Using Manual Loop.")
            scores = []
            u_vec = user_vector.flatten()
            for row in relevant_features:
                scores.append(np.dot(row, u_vec))
            scores = np.array(scores)

        relevant_venues['match_score'] = scores
        relevant_venues = relevant_venues.sort_values(
            by=['match_score', 'popularity_score'],
            ascending=[False, False]
        ).head(top_n)

        recommendations = []
        for _, row in relevant_venues.iterrows():
            rec = {
                "name": row['venue_name'],
                "category": row['category'],
                "price": row['price_tier'],
                "avg_cost": row.get('price_avg', 'N/A'),
                "rating": row.get('rating', 'N/A')
            }
            recommendations.append(rec)

        return recommendations

print("✅ RecommendationAgent class defined")

✅ RecommendationAgent class defined


## 🎯 Initialize Models

In [0]:

try:
    food_agent = FoodCostAgent(f"{MODELS_PATH}/food_cost_predictor_v1.pkl")
    print("✅ Food Agent initialized")
except Exception as e:
    print(f"⚠️ Food Agent failed: {e}")
    food_agent = None

try:
    lodging_agent = LodgingCostAgent(f"{MODELS_PATH}/lodging_cost_predictor_v1.pkl")
    print("✅ Lodging Agent initialized")
except Exception as e:
    print(f"⚠️ Lodging Agent failed: {e}")
    lodging_agent = None

try:
    entertainment_agent = EntertainmentAgent(
        f"{MODELS_PATH}/entertainment_cost_model.pkl",
        f"{DIRECTORY_PATH}/entertainment data/city_profiles.parquet"
    )
    print("✅ Entertainment Agent initialized")
except Exception as e:
    print(f"⚠️ Entertainment Agent failed: {e}")
    entertainment_agent = None

try:
    transport_agent = TransportAgent(
        f"{DIRECTORY_PATH}/transportation data/transportation_state.json",
        f"{DIRECTORY_PATH}/transportation data/transportation_data.csv"
        )
    print("✅ Transport Agent initialized")
except Exception as e:
    print(f"⚠️ Transport Agent failed: {e}")
    transport_agent = None

try:
    recommendation_agent = RecommendationAgent(f"{MODELS_PATH}/recommendation_engine.pkl")
    print("✅ Recommendation Agent initialized")
except Exception as e:
    print(f"⚠️ Recommendation Agent failed: {e}")
    recommendation_agent = None

print("\n" + "="*50)
print("🎉 Model initialization complete!")
print("="*50)

Loading Food Model from /tmp/models//food_cost_predictor_v1.pkl...
✅ Food Model loaded successfully.
✅ Food Agent initialized
Loading Lodging Model from /tmp/models//lodging_cost_predictor_v1.pkl...
✅ Lodging Model loaded. It knows 50 distinct states.
✅ Lodging Agent initialized
Loading Entertainment Agent from /tmp/models//entertainment_cost_model.pkl...
✅ Entertainment Model & Config loaded.
✅ Loaded city profiles for 634 cities.
✅ Entertainment Agent initialized
✅ State JSON loaded. Standardized 150 states.
✅ City CSV loaded. Standardized 600 profiles.
✅ Transport Agent initialized
  [ML] RecommendationAgent loaded. Matrix shape: (2425, 20)
✅ Recommendation Agent initialized

🎉 Model initialization complete!


## 🔧 Trip Calculation Function

In [0]:
def calculate_trip_cost(city, state, days, demographics, eating_out_ratio, 
                        cuisine, budget_level, interests):
    """
    Calculate trip costs using ML agents
    """
    estimates = {}
    travelers = sum(demographics.values())
    
    if travelers == 0:
        travelers = 1
        demographics["males"] = 1
    
    print(f"\n🔍 Calculating costs for {travelers} traveler(s) in {city}, {state}...\n")
    
    # Lodging Costs
    if lodging_agent:
        try:
            lodging_result = lodging_agent.predict_cost(
                state=state,
                travelers=travelers,
                nights=days,
                luxury_level=budget_level
            )
            estimates["Lodging"] = lodging_result["total_cost"]
            lodging_details = lodging_result["details"]
            print(f"🏨 Lodging: ${estimates['Lodging']:.2f}")
        except Exception as e:
            print(f"⚠️  Lodging calculation failed: {e}")
            estimates["Lodging"] = 150.0 * days * np.ceil(travelers / 2)
            lodging_details = {}
    else:
        estimates["Lodging"] = 150.0 * days * np.ceil(travelers / 2)
        lodging_details = {}
        
    # Food Costs
    if food_agent:
        try:
            food_result = food_agent.predict_cost(
                location=state,
                days=days,
                demographics=demographics,
                eating_out_ratio=eating_out_ratio,
                cuisine=cuisine,
                budget_level=budget_level
            )
            estimates["Food"] = food_result["total_cost"]
            food_options = food_result["food_options"]
            print(f"🍽️ Food: ${estimates['Food']:.2f}")
        except Exception as e:
            print(f"⚠️  Food calculation failed: {e}")
            estimates["Food"] = 50.0 * days * travelers
            food_options = {}
    else:
        estimates["Food"] = 50.0 * days * travelers
        food_options = {}

    # Entertainment Costs
    if entertainment_agent:
        try:
            ent_cost = entertainment_agent.predict_cost(
                city, state, travelers, days, budget_level=budget_level
            )
            estimates["Entertainment"] = ent_cost
            print(f"🎭 Entertainment: ${estimates['Entertainment']:.2f}")
        except Exception as e:
            print(f"⚠️  Entertainment calculation failed: {e}")
            estimates["Entertainment"] = 30.0 * days * travelers
    else:
        estimates["Entertainment"] = 30.0 * days * travelers
    
    # Transportation Costs
    if transport_agent:
        try:
            travel_style = 0 if "cheap" in budget_level.lower() else 2 if "luxury" in budget_level.lower() else 1
            lifestyle = "budget" if "cheap" in budget_level.lower() else "comfortable" if "luxury" in budget_level.lower() else "typical"
            
            transport_cost = transport_agent.predict_cost(
                city=city,
                state=state,
                days=days,
                travelers=travelers,
                travel_mode=travel_style,
                lifestyle=lifestyle
            )
            estimates["Transportation"] = transport_cost
            print(f"🚗 Transportation: ${estimates['Transportation']:.2f}")
        except Exception as e:
            print(f"⚠️  Transportation calculation failed: {e}")
            estimates["Transportation"] = 20.0 * days * travelers
    else:
        estimates["Transportation"] = 20.0 * days * travelers
    
    # Calculate total
    total_cost = sum(estimates.values())
    
    # Get recommendations
    recommendations = []
    if recommendation_agent:
        try:
            rec_prefs = {
                'interests': interests,
                'budget': budget_level,
                'max_spend': estimates.get('Entertainment', 100.0),
                'travelers': travelers
            }
            recommendations = recommendation_agent.recommend(city, state, rec_prefs, top_n=5)
        except Exception as e:
            print(f"⚠️  Recommendations failed: {e}")
    
    return {
        "total_cost": total_cost,
        "breakdown": estimates,
        "food_options": food_options,
        "lodging_details": lodging_details,
        "recommendations": recommendations
    }

print("✅ Trip calculation function defined")

✅ Trip calculation function defined


## 📊 Visualize Cost Breakdown Function
Creates a pie chart to visualize the cost distribution

In [0]:
import matplotlib.pyplot as plt
from colour import Color

def complementary_color(hex_color):
    c = Color(hex_color)
    c.hue = (c.hue + 0.5) % 1
    return c.hex_l


def visualize_cost_breakdown(result, city, state, days, budget_level):
    """
    Visualizes the cost breakdown of a trip.
    """

    # Prepare Data
    categories = list(result['breakdown'].keys())
    costs = list(result['breakdown'].values())

    # Optimized Color Mapping
    color_map = {
        'Lodging': '#3498db',
        'Food': '#e74c3c',
        'Entertainment': '#9b59b6',
        'Transportation': '#2ecc71',
        'Transport': '#2ecc71'
    }

    # Complementary colors
    color_map = {cat: complementary_color(color) for cat, color in color_map.items()}

    colors = [color_map.get(cat, '#f1c40f') for cat in categories]

    # Create Subplots (1 Row, 2 Columns)
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Bar Chart
    bars = axs[0].bar(categories, costs, color=colors, edgecolor='gray')
    axs[0].set_title('Cost Comparison ($)', fontsize=12, fontweight='bold')
    axs[0].set_ylabel('Cost ($)')

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        axs[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'${height:.0f}',
                    ha='center', va='bottom', fontsize=10)

    # Pie Chart
    axs[1].pie(costs, labels=categories, autopct='%1.1f%%', startangle=90, 
            colors=colors, shadow=True, explode=[0.05]*len(costs))
    axs[1].set_title('Cost Distribution (%)', fontsize=12, fontweight='bold')

    # Final Layout Adjustments
    plt.suptitle(f'Trip Analysis: {city}, {state} ({days} days)\nTotal Budget: ${result["total_cost"]:.2f}\nTravel Style: {budget_level}', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()

    # Display
    plt.show()

## 🚀 Trip Planning Function

In [0]:
import re


def show_results(city, state, days, males, females, children, eating_out_ratio, cuisine, budget_level, interests):
    """
    Shows the results of a trip calculation.
    """

    def is_valid_string(s):
        return isinstance(s, str) and bool(s.strip())
    
    def has_numbers(s):
        return any(char.isdigit() for char in s)
    
    def has_special_symbols(s):
        return bool(re.search(r'[^a-zA-Z0-9\s]', s))
    
    def is_valid(s):
        return is_valid_string(s) and not has_numbers(s) and not has_special_symbols(s)


    if not is_valid(city):
        print("Please provide a valid city.")
        return
    
    if not is_valid(state) or len(state) != 2 or not state.isalpha():
        print("Please provide a valid state.")
        return
    
    if not isinstance(days, int) or days <= 0:
        print("Please provide a valid duration.")
        return
    
    if not isinstance(males, int) or not isinstance(females, int) or not isinstance(children, int):
        print("Please provide valid numbers for demographics.")
        return
    
    if not isinstance(eating_out_ratio, int) or eating_out_ratio < 0 or eating_out_ratio > 100:
        print("Please provide a valid eating out ratio.")
        return
    
    if not is_valid(cuisine):
        print("Please provide a valid cuisine.")
        return
    
    # if not is_valid(interests):
    #     print("Please provide valid interests.")
    #     return
    
    if not is_valid(budget_level):
        print("Please provide a valid budget level.")
        return

    if males == 0 and females == 0 and children == 0:
        print("Please provide valid demographics.")
        return
    
    if males == 0 and females == 0:
        print("Children can't travel alone. Only with adults.")
        return

    state = state.upper()
    days = int(days)

    # Demographics
    demographics = {
        "males": males,
        "females": females,
        "children": children
    }

    # Food preferences
    dining_ratio = int(eating_out_ratio) / 100.0

    # Interests
    interests = [i.strip() for i in interests.split(",")]

    # Calculate trip
    print("="*70)
    print(f"🌎 AI TRAVEL AGENT - TRIP ESTIMATE")
    print("="*70)
    print(f"\n📍 Destination: {city}, {state}")
    print(f"📅 Duration: {days} days")
    # print(f"👥 Travelers: {sum(demographics.values())} ({demographics['males']}M, {demographics['females']}W, {demographics['children']} Kids)")
    print(f"👥 Travelers: {sum(demographics.values())} ({demographics['males']} {'Man' if demographics['males'] == 1 else 'Men'}, {demographics['females']} {'Woman' if demographics['females'] == 1 else 'Women'}, {demographics['children']} {'Kid' if demographics['children'] == 1 else 'Kids'})")

    print(f"💰 Budget Level: {budget_level}")
    print(f"🍽️ Dining: {int(dining_ratio*100)}% restaurants, {int((1-dining_ratio)*100)}% groceries")
    print(f"🎯 Interests: {', '.join(interests)}")
    print(f"🍴 Cuisine: {cuisine}")

    result = calculate_trip_cost(
        city=city,
        state=state,
        days=days,
        demographics=demographics,
        eating_out_ratio=dining_ratio,
        cuisine=cuisine,
        budget_level=budget_level,
        interests=interests
    )

    print("\n" + "="*70)
    print(f"💵 TOTAL ESTIMATED COST: ${result['total_cost']:.2f}")
    print("="*70)

    # Display cost breakdown
    print("\n📊 COST BREAKDOWN:")
    print("-" * 70)
    for category, cost in result['breakdown'].items():
        percentage = (cost / result['total_cost']) * 100
        print(f"  {category:15s}: ${cost:8.2f} ({percentage:5.1f}%)")

    # Display food options if available
    if result['food_options']:
        print("\n🍽️  ALTERNATIVE FOOD PLANS:")
        print("-" * 70)
        for option, cost in result['food_options'].items():
            print(f"  {option:20s}: ${cost:.2f}")

    # Display recommendations
    if result['recommendations']:
        print("\n🎯 TOP RECOMMENDED VENUES:")
        print("-" * 70)
        for i, rec in enumerate(result['recommendations'], 1):
            print(f"  {i}. {rec['name']}")
            print(f"     Category: {rec['category']} | Price: {rec['price']} | Rating: {rec.get('rating', 'N/A')}")
            if rec.get('avg_cost') != 'N/A':
                print(f"     Avg Cost: ${rec['avg_cost']}")
            print()

    print("="*70)
    print("✅ Trip planning complete!")
    print("="*70)

    visualize_cost_breakdown(result, city, state, days, budget_level)

## Travel Agent 🌎✈️

In [0]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Text Inputs
w_city = widgets.Text(value="New York", description="1. City")
w_state = widgets.Text(value="NY", description="2. State")

# Numeric Inputs
w_days = widgets.BoundedIntText(value=3, min=1, description="3. Days")

# Budget Dropdown
w_budget = widgets.Dropdown(
    options=["Cheap", "Moderate", "Luxury"],
    value="Moderate",
    description="4. Budget"
)

# Demographics (Numeric)
w_males = widgets.BoundedIntText(value=1, min=0, description="5. Men")
w_females = widgets.BoundedIntText(value=0, min=0, description="6. Women")
w_children = widgets.BoundedIntText(value=0, min=0, description="7. Children")

# Food Preferences
w_cuisine = widgets.Text(value="American", description="8. Cuisine")

w_dining_ratio = widgets.Dropdown(
    options=range(0, 110, 10),
    value=50,
    description="9. Dining %"
)

w_interests = widgets.Text(
    value="Tourism, Landmarks, Food",
    description="10. Interests"
)

# Create the Button and Output Area
run_btn = widgets.Button(
    description='Get Trip Cost',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to process the input values',
    icon='check'
)

# This "Output" widget captures print statements from the button click
out = widgets.Output()

# Define the Logic
def on_button_clicked(b):
    with out:
        clear_output() # Clear previous results

        show_results(
            city=w_city.value,
            state=w_state.value,
            days=w_days.value,
            budget_level=w_budget.value,
            males=w_males.value,
            females=w_females.value,
            children=w_children.value,
            eating_out_ratio=w_dining_ratio.value,
            cuisine=w_cuisine.value,
            interests=w_interests.value
        )

# Link the button to the function
run_btn.on_click(on_button_clicked)

# Stack inputs, button, and output area vertically
ui = widgets.VBox([
    widgets.HBox([w_city, w_state, w_days, w_budget]),
    widgets.HBox([w_males, w_females, w_children]),
    widgets.HBox([w_cuisine, w_dining_ratio, w_interests, run_btn]),
    # out
])

display(ui)
display(out)

Output()

### **🗺️ Travel Agent: Instructions for Use**

Before runnnig the notebook, please insert SAS Token in line 3 code cell 6

**Step 1: Configure Your Trip**
Use the interactive widgets below to define your trip parameters.
* **Destination:** Enter the `City` and the 2-letter `State` code (e.g., *NY*, *CA*).
* **Duration:** Set the number of `Days` for your trip.
* **Budget:** Select your preferred budget level (`Cheap`, `Moderate`, `Luxury`).
* **Travelers:** Enter the number of Males, Females, and Children in your group.
* **Preferences:**
    * Specify your favorite `Cuisine` (e.g., Italian, Mexican).
    * Set the `Dining %` slider (0% = All groceries, 100% = All restaurants).
    * List your `Interests` (e.g., Museums, Hiking, Nightlife).

**Step 2: Run the Analysis**
* After filling in the details, click the green **`🚀 Run Analysis`** button.
* The system will process your inputs through our ML models. *Please wait a moment for the results to generate.*

**Step 3: Review Your Itinerary**
* Scroll down to view the **Trip Report**, which includes:
    * Total Estimated Cost
    * Daily Itinerary Breakdown
    * Recommended Venues & Activities
    * Visual Cost Breakdown

> **⚠️ Note:** Ensure your cluster is running before interacting with the widgets. If the widgets do not appear, try refreshing the browser tab or rerunning the last code cell.

---